Read Forty Datasets

In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
import re  

# Load the Excel file
datasets_xlsx = pd.read_excel("FortyDatasets.xlsx")


Clean Columns

In [2]:
import pandas as pd

def clean_columns(df):
    # Extract the ID from the column name
    df['ID'] = df['Original Column'].str.extract(r'(\d+)\.')
    
    # Extract the Column name
    df['Column'] = df['Original Column'].str.extract(r'\d+\.\s*([^/(:]+)')
    
    # Extract the remaining part for description splitting
    df['Descriptions'] = df['Original Column'].str.extract(r'\d+\.\s*[^/(:]+\s*(.*)')[0].str.strip(': ')
    
    # Replace symbols (, ), :, and / with : and remove duplicate ::
    df['Descriptions'] = df['Descriptions'].str.replace('[\(:/\)]', ':', regex=True).str.replace('::', ':', regex=True)

    # Initialize a list to store split descriptions
    split_descriptions = []
    
    for desc in df['Descriptions']:
        # If description is empty, continue with an empty list
        if not desc:
            split_descriptions.append([None])
            continue
        
        # Split the description by ":"
        parts = [x.strip() for x in desc.split(':') if x.strip()]
        
        split_descriptions.append(parts)
        
    # Convert the list of lists into a DataFrame
    split_descriptions_df = pd.DataFrame(split_descriptions)
    
    # Assign the multiple description fields
    for i in range(split_descriptions_df.shape[1]):
        col_name = f'Description {i + 1}'
        df[col_name] = split_descriptions_df[i]
    
    # Trim leading/trailing white space
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Replace empty strings with None
    df = df.replace(r'^\s*$', None, regex=True)
    
    # Drop the temporary 'Descriptions' column
    df = df.drop(columns=['Descriptions'])
    
    return df

# Test the function
test_df = pd.DataFrame({
    'Original Column': [
        '14. num (the predicted attribute)',
        '1. lettr: capital letter (26 values from A to Z)',
        '2. x-box: horizontal position of the box (integer)',
        '1. sepal length in cm',
        '20. Foreign worker (qualitative)',
        '12. A12: Categorical with values: t, f',
        '55. capital_run_length_average (1 continuous real attribute): Average length of uninterrupted sequences of capital letters',
        '1. Class Name (party affiliation): democrat, republican',
        '1.Sex / nominal / -- / M, F, and I (infant)'
    ]
})

cleaned_df = clean_columns(test_df)
cleaned_df


,Original Column,ID,Column,Description 1,Description 2,Description 3,Description 4
0,14. num (the predicted attribute),14,num,the predicted attribute,None,None,None
1,1. lettr: capital letter (26 values from A to Z),1,lettr,capital letter,26 values from A to Z,None,None
2,2. x-box: horizontal position of the box (inte...,2,x-box,horizontal position of the box,integer,None,None
3,1. sepal length in cm,1,sepal length in cm,None,None,None,None
4,20. Foreign worker (qualitative),20,Foreign worker,qualitative,None,None,None
5,"12. A12: Categorical with values: t, f",12,A12,Categorical with values,"t, f",None,None
6,55. capital_run_length_average (1 continuous r...,55,capital_run_length_average,1 continuous real attribute,Average length of uninterrupted sequences of c...,None,None
7,"1. Class Name (party affiliation): democrat, r...",1,Class Name,party affiliation,"democrat, republican",None,None
8,"1.Sex / nominal / -- / M, F, and I (infant)",1,Sex,nominal,--,"M, F, and I",infant


In [3]:
def preprocess_columns(df):
    # Only convert "Column" to string type for non-null values and then to lowercase to create "CleanedColumn"
    mask = df['Column'].notna()
    df.loc[mask, 'CleanedColumn'] = df.loc[mask, 'Column'].astype(str).str.lower()
    
    # Remove '-' and '_' characters only for non-null values
    mask = df['CleanedColumn'].notna()
    df.loc[mask, 'CleanedColumn'] = df.loc[mask, 'CleanedColumn'].str.replace('[-_]', ' ', regex=True)
 
    return df

# Load the Excel file containing column names
columns_xlsx = pd.read_excel("AllColumnsFromFortyDatasets.xlsx")

# Clean the columns
cleaned_columns_df = clean_columns(columns_xlsx.copy())

# Further cleaning on 'Description 2'
cleaned_columns_df['Description 2'] = cleaned_columns_df['Description 2'].str.split('(').str[0].str.strip()

# Preprocess the column names
preprocessed_columns_df = preprocess_columns(cleaned_columns_df)

preprocessed_columns_df



,index,name,area,Original Column,ID,Column,Description 1,Description 2,Description 3,Description 4,Description 5,Description 6,Description 7,CleanedColumn
0,52,Iris,Life,1. sepal length in cm,1,sepal length in cm,None,None,None,None,None,None,None,sepal length in cm
1,52,Iris,Life,2. sepal width in cm,2,sepal width in cm,None,None,None,None,None,None,None,sepal width in cm
2,52,Iris,Life,3. petal length in cm,3,petal length in cm,None,None,None,None,None,None,None,petal length in cm
3,52,Iris,Life,4. petal width in cm,4,petal width in cm,None,None,None,None,None,None,None,petal width in cm
4,52,Iris,Life,5. class,5,class,None,None,None,None,None,None,None,class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,13,Balloons,Social,"1.Color: yellow, purple",1,Color,"yellow, purple",None,None,None,None,None,None,color
943,13,Balloons,Social,"2.size: large, small",2,size,"large, small",None,None,None,None,None,None,size
944,13,Balloons,Social,"3.act: stretch, dip",3,act,"stretch, dip",None,None,None,None,None,None,act
945,13,Balloons,Social,"4.age: nominal adult, child",4,age,"nominal adult, child",None,None,None,None,None,None,age


Open formats and abreviations dictionaries

In [4]:
dictionary = {}

# Open the file and read line by line
with open("formats_dictionary.txt", "r") as file:
    for line in file:
        # Remove the trailing newline and comma, then split the line into key and value at the colon
        key, value = line.rstrip(",\n").split(":")
    
        # Remove the quotes around the key and value
        key = key.strip("'")
        value = value.strip("'")

        # Add the key-value pair to the dictionary
        dictionary[key] = value

abbreviations_dict = {}
with open("abbreviations_dictionary.txt", "r") as file:
    for line in file:
        abbr, full_form = line.strip().split(":")
        abbreviations_dict[abbr.strip()] = full_form.strip()

Replace abbreviations

In [5]:
def replace_abbreviations(text, abbreviations_dict):
    words = text.split()
    replaced_words = [abbreviations_dict.get(word, word) for word in words]
    return " ".join(replaced_words)

# Extract unique target words and their corresponding analysis results
#target_words_analysis = analysis_xlsx[['ColumnKeyword', 'ColumnFormat']].dropna().drop_duplicates()
#description_words_analysis = analysis_xlsx[['DescriptionKeyword', 'DescriptionFormat']].dropna().drop_duplicates()

target_words_dict = dictionary

# Create dictionaries for mapping
#target_words_dict = dict(zip(target_words_analysis['ColumnKeyword'], target_words_analysis['ColumnFormat']))
#description_words_dict = dict(zip(description_words_analysis['DescriptionKeyword'], description_words_analysis['DescriptionFormat']))
description_words_dict = dictionary


Apply analysis

In [6]:
import pandas as pd
import re

def apply_analysis(df, target_words_dict, description_words_dict, abbreviations_dict):
    # Initialize new columns
    df['ColumnKeyword'] = None
    df['ColumnFormat'] = None
    df['DescriptionKeyword'] = None
    df['DescriptionFormat'] = None

    # Apply target words analysis
    for i, row in df.iterrows():
        std_col_name = row['CleanedColumn']
        col_name = row['Column']

    # Skip if 'CleanedColumn' is missing
        if pd.isnull(std_col_name):
            continue
        
        # Pattern for matching 'ID' (in uppercase) at the end of a column name
        id_pattern_upper = r'ID$'

        # Check if the column name ends with 'ID' in uppercase
        if re.search(id_pattern_upper, col_name):
            df.at[i, 'ColumnKeyword'] = 'id'
            df.at[i, 'ColumnFormat'] = target_words_dict.get('id')
            continue

        found = False

        for word, analysis in target_words_dict.items():
            # Special handling for 'name'
            if word == 'name':
                pattern = rf'({word})(?![\w-])'
            else:
                # General matching for other terms
                pattern = rf'\b{word}\b'
            
            if re.search(pattern, std_col_name, re.IGNORECASE):
                df.at[i, 'ColumnKeyword'] = word
                df.at[i, 'ColumnFormat'] = analysis
                found = True
                break

        # If no match found, replace abbreviations and try again
        if not found:
            replaced_text = replace_abbreviations(row['CleanedColumn'], abbreviations_dict)
            for word, analysis in target_words_dict.items():
                if re.search(rf'\b{word}\b', replaced_text, re.IGNORECASE):
                    df.at[i, 'ColumnKeyword'] = word
                    df.at[i, 'ColumnFormat'] = analysis
                    found = True
                    break

        # If still no match, break down the word into substrings
        if not found:
            for j in range(len(row['CleanedColumn']), 2, -1):
                for k in range(len(row['CleanedColumn']) - j + 1):
                    subword = row['CleanedColumn'][k:k+j]
                    
                    # Check if the subword exists in the abbreviation dictionary
                    expanded_subword = abbreviations_dict.get(subword, None)
                    if expanded_subword:
                        # If the expanded subword exists in the target words dictionary, use it
                        if expanded_subword in target_words_dict:
                            df.at[i, 'ColumnKeyword'] = expanded_subword
                            df.at[i, 'ColumnFormat'] = target_words_dict[expanded_subword]
                            found = True
                            break
                        
                    # Else, continue with the original subword
                    elif subword in target_words_dict:
                        df.at[i, 'ColumnKeyword'] = subword
                        df.at[i, 'ColumnFormat'] = target_words_dict[subword]
                        break

                if found:
                    break

    # Apply description words analysis
    for i, row in df.iterrows():
        # Skip if 'Description 1' and 'Description 2' are missing
        if pd.isnull(row['Description 1']) and pd.isnull(row['Description 2']):
            continue

        for word, analysis in description_words_dict.items():
            if ((not pd.isnull(row['Description 1']) and re.search(rf'\b{word}\b', row['Description 1'], re.IGNORECASE)) or 
               (not pd.isnull(row['Description 2']) and re.search(rf'\b{word}\b', row['Description 2'], re.IGNORECASE))):
                df.at[i, 'DescriptionKeyword'] = word
                df.at[i, 'DescriptionFormat'] = analysis
                break

    return df

# Apply this function to the dataframe
analysed_columns_df = apply_analysis(preprocessed_columns_df.copy(), target_words_dict, description_words_dict, abbreviations_dict)

In [7]:
import pandas as pd
import re

def apply_analysis(df, target_words_dict, description_words_dict, abbreviations_dict):
    # Presuming 'formats_ordered_list' is a list of dictionary keys in the order they appear in 'formats_dictionary'
    formats_ordered_list = list(target_words_dict.keys())

    # Initialize new columns
    df['ColumnKeyword'] = None
    df['ColumnFormat'] = None
    df['DescriptionKeyword'] = None
    df['DescriptionFormat'] = None

    # Apply target words analysis
    for i, row in df.iterrows():
        std_col_name = row['CleanedColumn']
        col_name = row['Column']

        # Skip if 'CleanedColumn' is missing
        if pd.isnull(std_col_name):
            continue

        found = False

        # Iterate through each word based on the order in 'formats_ordered_list'
        for word in formats_ordered_list:
            analysis = target_words_dict[word]
            # Special handling for 'name'
            if word == 'name':
                pattern = rf'({word})(?![\w-])'
            # Special handling for uppercase 'ID' at the end of a column name
            elif word == 'id' and col_name.endswith('ID'):
                df.at[i, 'ColumnKeyword'] = 'id'
                df.at[i, 'ColumnFormat'] = target_words_dict.get('id', 'ID column')
                found = True
                break
            else:
                # General matching for other terms
                pattern = rf'\b{word}\b'

            # Search for the pattern in the CleanedColumn
            if re.search(pattern, std_col_name, re.IGNORECASE):
                df.at[i, 'ColumnKeyword'] = word
                df.at[i, 'ColumnFormat'] = analysis
                found = True
                break

        # If no match found, replace abbreviations and try again
        if not found:
            replaced_text = replace_abbreviations(std_col_name, abbreviations_dict)
            for word in formats_ordered_list:
                pattern = rf'\b{word}\b'
                if re.search(pattern, replaced_text, re.IGNORECASE):
                    df.at[i, 'ColumnKeyword'] = word
                    df.at[i, 'ColumnFormat'] = target_words_dict[word]
                    found = True
                    break

        # If still no match, break down the word into substrings
        if not found:
            for j in range(len(row['CleanedColumn']), 2, -1):
                for k in range(len(row['CleanedColumn']) - j + 1):
                    subword = row['CleanedColumn'][k:k+j]
                    
                    # Check if the subword exists in the abbreviation dictionary
                    expanded_subword = abbreviations_dict.get(subword, None)
                    if expanded_subword:
                        # If the expanded subword exists in the target words dictionary, use it
                        if expanded_subword in target_words_dict:
                            df.at[i, 'ColumnKeyword'] = expanded_subword
                            df.at[i, 'ColumnFormat'] = target_words_dict[expanded_subword]
                            found = True
                            break
                        
                    # Else, continue with the original subword
                    elif subword in target_words_dict:
                        df.at[i, 'ColumnKeyword'] = subword
                        df.at[i, 'ColumnFormat'] = target_words_dict[subword]
                        break

                if found:
                    break

    # Apply description words analysis
    for i, row in df.iterrows():
        # Skip if 'Description 1' and 'Description 2' are missing
        if pd.isnull(row['Description 1']) and pd.isnull(row['Description 2']):
            continue

        for word, analysis in description_words_dict.items():
            if ((not pd.isnull(row['Description 1']) and re.search(rf'\b{word}\b', row['Description 1'], re.IGNORECASE)) or 
               (not pd.isnull(row['Description 2']) and re.search(rf'\b{word}\b', row['Description 2'], re.IGNORECASE))):
                df.at[i, 'DescriptionKeyword'] = word
                df.at[i, 'DescriptionFormat'] = analysis
                break

    return df

# Apply this function to the dataframe
analysed_columns_df = apply_analysis(preprocessed_columns_df.copy(), target_words_dict, description_words_dict, abbreviations_dict)

FinalFormat

In [8]:
def analysis_of_column(std_col_analysis, desc_found_analysis, index_value):
    if std_col_analysis is None and desc_found_analysis is None:
        return 'NaN'
    
    # If one is 'string' and the other is not, choose the one that is not 'string'
    if std_col_analysis == 'string' and desc_found_analysis not in [None, 'string', 'NaN']:
        return desc_found_analysis
    if desc_found_analysis == 'string' and std_col_analysis not in [None, 'string', 'NaN']:
        return std_col_analysis
    
    # If either is 'None', choose the one that has a value
    if pd.isnull(std_col_analysis):
        return desc_found_analysis
    elif pd.isnull(desc_found_analysis):
        return std_col_analysis

    # If both are 'numerical' or if std_col_analysis is in the predefined list, return std_col_analysis
    if (std_col_analysis.startswith("numerical") and desc_found_analysis.startswith("numerical")) or \
       std_col_analysis in ["phone", "ID column", "percentage", "month", "datetime", "date",
                            "weekday", "week", "country", "city", "state", "year", "name",
                            "latitude", "longitude", "postal code", "URL format","IP format", "E-mail format", "binary"]:
        return std_col_analysis
    elif std_col_analysis in ["age"] and desc_found_analysis not in ['categorical']:
        return std_col_analysis
    # If no exceptions apply, return the value from "DescriptionFormat"
    else:
        return desc_found_analysis

# Apply the function to create the new column
analysed_columns_df['FinalFormat'] = analysed_columns_df.apply(
    lambda row: analysis_of_column(
        row['ColumnFormat'],
        row['DescriptionFormat'],
        row['index']),  # Make sure 'index' column exists in your DataFrame
    axis=1
)


In [9]:
def identify_origin(row):
    analysis_col = row['FinalFormat']
    std_col_analysis = row['ColumnFormat']
    desc_found_analysis = row['DescriptionFormat']

    # Check if the value in 'FinalFormat' came from 'ColumnFormat'
    if pd.notnull(std_col_analysis) and analysis_col == std_col_analysis:
        return row['ColumnKeyword']

    # Check if the value in 'FinalFormat' came from 'DescriptionFormat'
    elif pd.notnull(desc_found_analysis) and analysis_col == desc_found_analysis:
        return row['DescriptionKeyword']

    # If none of the above conditions are met, return NaN or any default value you prefer
    else:
        return 'NaN'

# Create the new column 'SourceKeyword' in analysed_columns_df
analysed_columns_df['SourceKeyword'] = analysed_columns_df.apply(identify_origin, axis=1)


Save results

In [10]:
# Save the results to a new Excel file
analysed_columns_df.to_excel("AnalysedColumns.xlsx", index=False)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Last run on: 2024-04-03 13:37:04
